In [45]:
campaign_id = 1
manager_ethereum_address = '0xD3e9D60e4E4De615124D5239219F32946d10151D'

In [46]:
import base64
import mysql.connector as mysql
import datetime
import pandas as pd
import time

import json
from web3 import Web3 # https://web3py.readthedocs.io/en/stable/contracts.html#contract-functions

#######
## INIT
#######

# load api key
secret = {}
with open('secret_staging.txt') as f:
    lines = f.readlines()
    for line in lines:
        secret[line.split("=")[0]] = line.split("=")[1].replace("\n","")

# init web3
infura_url = secret['INFURAURL1']
web3 = Web3(Web3.HTTPProvider(infura_url))
print(f"Connected to infura: {infura_url}")

# load abi
with open('abi.json') as f:
    abi = json.load(f)
# load bytecode
bytecode = '...'

contract_address = '0xe7b8897D824C78e5C1076023508C43Eace3D2Afc'

Connected to infura: https://kovan.infura.io/v3/1070d8486bc64afdb9c730d6f2850d6c


In [55]:
###########################
## GET USER:WALLET MAPPINGS
###########################

# convert rewards pending twitter handles to wallet ids
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT twitter_handle, ethereum_address FROM users;'
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
db.close()
# convert to dataframe
users = []
for record in records:
    users.append(dict(zip(['twitter_handle', 'ethereum_address'], record)))
# convert to dataframe and lowercase handle
users = pd.DataFrame(users)
users['twitter_handle'] = users['twitter_handle'].str.lower()

# create dicts
user_wallets = dict(zip(list(users['twitter_handle']), users['ethereum_address']))
wallets_users = dict(zip(list(users['ethereum_address']), users['twitter_handle']))

# create list
user_list = list(user_wallets.keys())

In [48]:
##################################################
### get all rewards for this campaign as dataframe
##################################################
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT campaign_id, twitter_handle FROM rewards WHERE blockchain_write_time is NULL;'
cursor.execute(query)
records_rewards = cursor.fetchall()
cursor.close()
db.close()
# convert to dataframe
rewards = []
for record in records_rewards:
    rewards.append(dict(zip(['campaign_id', 'twitter_handle'], record)))
df = pd.DataFrame(rewards)
print(f"DEGBUG: LEN DF {len(df)}")

DEGBUG: LEN DF 102


In [75]:
###################################################
### GET LIST OF WALLETS TO REWARD FOR THIS CAMPAIGN
###################################################

# list of handles to reward
rewards_pending = list(df[df['campaign_id'] == campaign_id]['twitter_handle'])

# convert from twitter handles to wallets
rewards_pending = [user_wallets[r] for r in rewards_pending]

In [76]:
# # # MANUAL OVERRIDE FOR DEBUGGING - COMMENT OUT BELOW FOR REAL SC WRITE
# rewards_pending = []
# # # rewards_pending.append('0xAf84db74fa05825B04D3a7eF4cFB98BF1d381C80') # granddanois3
# # # rewards_pending.append('0x3708045C19c4D0eaB69289F8BC10D4a1fF4EDc6a') # archipelagoai
# # # rewards_pending.append('0xe3f201d68B473c89b3D82eA9c9b0E951DB7345A8') # sebastianstoll0
# rewards_pending.append('0x0e8b5fD2F79F125fBfE3f2eE5C9De9211694A269') # alxcnwy

In [78]:
for i, reward_pending in enumerate(rewards_pending):
    print(f"Creating SC tx {i+1}/{len(rewards_pending)} to reward {reward_pending}")

    ###################################
    ### WRITE REWARDS TO SMART CONTRACT
    ###################################
    contract = web3.eth.contract(abi=abi, bytecode=bytecode)

    tx = contract.functions.rewardAddresses(manager_ethereum_address, campaign_id, [reward_pending]).buildTransaction(
        {'gas':500000, 
         'gasPrice': web3.toWei('24', 'gwei'),
         'from': secret['ETHBACKENDPUBLIC'],
         'to': contract_address,
         'nonce': web3.eth.getTransactionCount(secret['ETHBACKENDPUBLIC'])
        })
    
    signed_txn = web3.eth.account.signTransaction(tx, private_key=secret['ETHBACKENDPRIVATE'])
    tx_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
    print(f"TX hash: {tx_hash.hex()}")
    receipt = web3.eth.waitForTransactionReceipt(tx_hash, timeout=600)
    print(receipt)

    if receipt['status'] == 1:
        print(f"SUCCESS writing rewards to SC for campaign {campaign_id} tx: {tx_hash.hex()}")

        # update blockchain_write_time
        db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
        cursor = db.cursor()
        query = "UPDATE rewards SET blockchain_write_time=%s, blockchain_write_tx_hash=%s, gas_used=%s, blockchain_write_tx_status=%s WHERE campaign_id=%s AND manager_ethereum_address=%s AND twitter_handle=%s;"
        values = (str(datetime.datetime.now()).split('.')[0], str(tx_hash.hex()), receipt['gasUsed'], 1, str(campaign_id), manager_ethereum_address, wallets_users[reward_pending])
        cursor.execute(query, values)
        db.commit()
        print(cursor.rowcount, "record updated")
        cursor.close()
        db.close()  
    else:
        print(f"ERROR FAIL writing rewards to SC for campaign {campaign_id} tx: {tx_hash.hex()}")

Creating SC tx 1/102 to reward 0x93959c91653F8Cf587A5253ec82c16875012e526
TX hash: 0x03faf117ef57b939f1c0f4cb288414e195d1ea8ddeba9536a760523bb7fd8d1d
AttributeDict({'blockHash': HexBytes('0x443a1cca2ffa1ed53e75dd60fc05c370550d3e73009e28d02b876b4edbfb5a5d'), 'blockNumber': 22328779, 'contractAddress': None, 'cumulativeGasUsed': 132220, 'from': '0xe9540071278Dba4374A42a84B48c5e230328AeF7', 'gasUsed': 22220, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': '0xe7b8897D824C78e5

1 record updated
Creating SC tx 8/102 to reward 0x62661B101ca48734668669a9F1Cb83c4889049F3
TX hash: 0x77b5cac5c28dda035b40fe260675b0b1a366f4a43a82b63a8c11a1c4aeb8cf52
AttributeDict({'blockHash': HexBytes('0xe8481db75d717b2a170577892ddfb51252f2d26b2bf00bcb3b8f9545506349fe'), 'blockNumber': 22328789, 'contractAddress': None, 'cumulativeGasUsed': 22220, 'from': '0xe9540071278Dba4374A42a84B48c5e230328AeF7', 'gasUsed': 22220, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': '0x

1 record updated
Creating SC tx 15/102 to reward 0x91212de083d29342F670FB69Ffa68176369e3B20
TX hash: 0x903fccdeb4ce98197d2b0a2fcfd84686d511fadd369ae86bb6242d2c1003cfdc
AttributeDict({'blockHash': HexBytes('0x8c333851dbf3766b5f440d3deecedeee4029dda6ea914584b3963238139ba87f'), 'blockNumber': 22328802, 'contractAddress': None, 'cumulativeGasUsed': 22220, 'from': '0xe9540071278Dba4374A42a84B48c5e230328AeF7', 'gasUsed': 22220, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': '0

KeyboardInterrupt: 

# Check confirmation status

In [59]:
##############################################################
### check confirmation status for all unconfirmed transactions
##############################################################

# get all unconfirmed txs
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT blockchain_write_tx_hash FROM rewards WHERE blockchain_write_tx_status = 0;'
cursor.execute(query)
records_txs = cursor.fetchall()
cursor.close()
db.close()
# convert to dataframe
txs = []
for record in records_txs:
    txs.append(record[0])
print(f"DEGBUG: UNCONFIRMED TXS {len(txs)}")

# check tx status and update db if confirmed
for tx in txs:
    receipt = web3.eth.getTransactionReceipt(tx)
    
    if receipt['status'] == 1:
        # update blockchain_write_time
        db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
        cursor = db.cursor()
        query = "UPDATE rewards SET blockchain_write_time=%s, blockchain_write_tx_status=%s, gas_used=%s WHERE campaign_id=%s AND manager_ethereum_address=%s AND twitter_handle=%s;"
        values = (str(datetime.datetime.now()).split('.')[0], 1, receipt['gasUsed'], str(campaign_id), manager_ethereum_address, wallets_users[reward_pending])
        cursor.execute(query, values)
        db.commit()
        print(cursor.rowcount, "record updated")
        cursor.close()
        db.close() 
    else:
        print(f"ERROR - {tx} receipt status != 1")
        print(receipt)

Rewarding 1/1 0xe9540071278Dba4374A42a84B48c5e230328AeF7
TX hash: 0xb1cee719715c454467106ffc2c6b73c6234b74f7b5486bf84c336fb49b70e618
AttributeDict({'blockHash': HexBytes('0xffb054245778a31764542448d8fb31a865865ad0cadf4a5465ee2f1ac315d557'), 'blockNumber': 22328506, 'contractAddress': None, 'cumulativeGasUsed': 22208, 'from': '0xe9540071278Dba4374A42a84B48c5e230328AeF7', 'gasUsed': 22208, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': '0xe7b8897D824C78e5C1076023508C43Eace

# Estimate gas

In [57]:
contract.address = contract_address

In [58]:
contract.address

'0xe7b8897D824C78e5C1076023508C43Eace3D2Afc'

In [59]:
tx = contract.functions.rewardAddresses(ids_manageraddresses[campaign_id], int(campaign_id), rewards_pending).buildTransaction(
    {'gas':3000000, 
     'gasPrice': web3.toWei('23', 'gwei'),
     'from': secret['ETHBACKENDPUBLIC'],
     'to': contract_address,
     'nonce': web3.eth.getTransactionCount(secret['ETHBACKENDPUBLIC'])
    })


In [62]:
tx.estimateGas()

AttributeError: 'dict' object has no attribute 'estimateGas'

In [61]:
contract

web3._utils.datatypes.Contract

In [ ]:
 web3.eth.estimateGas({
     "from"      : walletbase,       
     "nonce"     : value, 
     "to"        : contractAddr,     
     "data"      : data
})

In [70]:
contract.address

'0xe7b8897D824C78e5C1076023508C43Eace3D2Afc'

In [71]:
contract.functions.rewardAddresses(ids_manageraddresses[campaign_id], int(campaign_id), rewards_pending).estimateGas(    {'gas':3000000, 
     'gasPrice': web3.toWei('23', 'gwei'),
     'from': secret['ETHBACKENDPUBLIC'],
     'to': contract_address,
     'nonce': web3.eth.getTransactionCount(secret['ETHBACKENDPUBLIC'])
    })

# https://github.com/ethereum/web3.py/blob/master/web3/contract.py

ValueError: Cannot set to in estimateGas transaction